In [ ]:
import torch
from diffusers import DiffusionPipeline, DDIMScheduler
import numpy as np
import utils
import os
import cv2
import diffusers
from PIL import Image

print("diffusers:",diffusers.__version__)


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
MY_TOKEN = ''
LOW_RESOURCE = False
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 7.5
MAX_NUM_WORDS = 77
device = torch.device('cuda:4') if torch.cuda.is_available() else torch.device('cpu')
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
ldm_stable = DiffusionPipeline.from_pretrained("./models--stabilityai--stable-diffusion-xl-base-1.0",scheduler = scheduler).to(device)
ldm_stable.enable_vae_tiling()
tokenizer = ldm_stable.tokenizer
output_folder = "./modify_by_reference/experiments"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [4]:
def run_and_display(prompts, in_img = None, full_img = None, generator=None):

    images = utils.text2image_ldm_stable(ldm_stable, prompts, real_img=in_img, full_img = full_img, num_inference_steps=NUM_DIFFUSION_STEPS, generator=generator)

    return images

In [ ]:
g_cpu = torch.Generator().manual_seed(8888)
start_idx = 0
step_len = 4

dir_coco = "./out_img"
out_dix = "./modify_by_reference_SDXL/inversion_exp_out"
method = "/tst"

out_folder = out_dix+method
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

total_img_num = len(os.listdir(dir_coco))
print("total amount of image is",total_img_num)

name_idx = start_idx
while name_idx < total_img_num:
    out_path = out_folder + "/" + str(name_idx) + ".jpeg"
    if os.path.exists(out_path):
        name_idx = name_idx + step_len
        continue

    real_img = cv2.imread(dir_coco + "/" + str(name_idx) + ".jpg")
    real_img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)

    real_img_full = np.array(Image.fromarray(real_img).resize((1024,1024))) 

    prompts = [""]

    image = run_and_display(prompts, generator=g_cpu, full_img=real_img_full)

    image = np.squeeze(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGRA) 
    cv2.imwrite(out_path,image)
    torch.cuda.empty_cache()
    name_idx = name_idx + step_len